### Comparing methods


In [2]:
import numpy as np 
import importlib
import testDijkToEdge
import maxProbAstar
import path_evaluation
from maxProbAstar import MaxProbAstar
from problem_setting import ProblemSetting
from path_evaluation import PathEvaluation
import copy
import math
import matplotlib.pyplot as plt
import time
import automaton_dp
import plot3d_fire
import pickle
import seaborn as sns
import os
import functools
from testDijkToEdge import UpdateEnv

In [168]:
# problem parameters
_SIZE = 20
_STPFlag = 1 # flag for enabling STP. If set to 0, use average probability
_obsvEffectRange=2
_replanRange=2

In [328]:
# problem setup
xTar = 8
yTar = 16
xEnd = 11
yEnd = 19
p1 = ProblemSetting(target = [[xTar,yTar]], endPoint = [[xEnd,yEnd]],_stochastic_environment_flag=1, _setting_num=1)
# monteCarloAverageFireMap, monteCarloFireMap = p1.compute_monteCarlo_2(3000)
monteCarloFireMap = pickle.load(open('MCFMs3000',"rb"))
monteCarloAverageFireMap = pickle.load(open('MCAFMs3000',"rb"))

In [357]:
# DP solution
# aDP1 = automaton_dp.AutomatonDP(p1.M,p1.N,p1.T,p1.Map,monteCarloAverageFireMap, monteCarloFireMap, p1.StartPoint,p1.EndPoint,p1.Target,p1.Wall,p1.Fire,Q=4, STPFlag=_STPFlag)
# aDP1.solve()
# with open('aDP1','wb') as fp:
#         pickle.dump(aDP1, fp)
aDP1 = pickle.load(open('aDP1',"rb"))

In [362]:
# query old path
oldCost, _oldPath, oldControl = aDP1.query_optimal_solution(0, [8,0,-1])
_oldPathCopy = copy.deepcopy(_oldPath)
aDP1.remove_duplicated_path_segs(_oldPathCopy)
oldPath = _oldPathCopy[7:]
print(oldPath)

[(8, 7, -1), (7, 7, -1), (7, 8, -1), (6, 8, -1), (6, 9, -1), (6, 10, -1), (5, 10, -1), (5, 11, -1), (5, 12, -1), (5, 13, -1), (5, 14, -1), (5, 15, -1), (5, 16, -1), (6, 16, -1), (7, 16, -1), (8, 16, -2), (8, 17, -2), (8, 18, -2), (8, 19, -2), (9, 19, -2), (10, 19, -2), (11, 19, -3)]


In [332]:
# reload stuff
importlib.reload(path_evaluation)
from path_evaluation import PathEvaluation
importlib.reload(maxProbAstar)
from maxProbAstar import MaxProbAstar
importlib.reload(testDijkToEdge)
from testDijkToEdge import UpdateEnv

# let the robot observe at xT=8,yT=7, at tT = 7
tT = 7; xT = 8; yT = 7; # s=138 for (7,9), 7 for (7,7) # 653 for lots
T=tT
upe = UpdateEnv(p1.M,p1.N,p1.Wall,p1.Fire,monteCarloFireMap, monteCarloAverageFireMap, obsvEffectRange=_obsvEffectRange, replanRange=_replanRange,tT = tT,aDP1=aDP1)

In [323]:
importlib.reload(problem_setting)
from problem_setting import ProblemSetting

In [369]:
fSet = [[[6,6]],[[6,7]],[[6,8]],[[6,9]],[[7,9]],[[9,9]],[[10,9]]] #observed fire # REMEMBER THIS>>>>
# ,[[10,8]],[[10,7]],[[10,6]]
# fSet = [[[6,6]]]
newPathList = []
oldSafeProbList = []
newSafeProbList = []
p3 = ProblemSetting(target = [[xTar,yTar]], endPoint = [[xEnd,yEnd]],_stochastic_environment_flag=1, _setting_num=1)
upe = UpdateEnv(p1.M,p1.N,p1.Wall,p1.Fire,monteCarloFireMap, monteCarloAverageFireMap, obsvEffectRange=_obsvEffectRange, replanRange=_replanRange,tT = tT,aDP1=aDP1)
for f in fSet:   
    upe.observe_from_fire(f,tT,xT,yT)
    _t = time.time()
    upe.dijk_to_edge(upe.replanRange)
    maxNeighborIndex = np.argmax(np.exp(np.multiply(-1,upe.fullPossibleProbs)))
    newPath = copy.deepcopy(upe.possiblePathsAfter[maxNeighborIndex])
    for p in range(len(upe.possiblePathsBefore[maxNeighborIndex])-2,-1,-1):
        newPath.insert(0,(upe.possiblePathsBefore[maxNeighborIndex][p][0],upe.possiblePathsBefore[maxNeighborIndex][p][1],-1)) # NOTICE: only -1 for now
    newPathList.append(newPath)
    elapsed_ = time.time() - _t
    print('Replanning took: %f seconds' %(elapsed_))
    monteCarloAverageFireMapExamObsv, monteCarloFireMapExamObsv = p3.compute_monteCarlo_obsv(200,pickleSave=True,fireLocationArray=f,obsvTime=tT,fileName=str(f[0][0])+str(f[0][1]))
    pe3 = PathEvaluation(monteCarloFireMapExamObsv)
    newSafeProbList.append(pe3.evaluate_path_t(newPath, tT))
    oldSafeProbList.append(pe3.evaluate_path_t(oldPath, tT))

Replanning took: 43.665324 seconds
counter is 2199
h is 200
Replanning took: 42.375632 seconds
counter is 2363
h is 200
math domain error
math domain error
Replanning took: 40.967614 seconds
counter is 3523
h is 200
math domain error
math domain error
math domain error
math domain error
Replanning took: 38.091253 seconds
counter is 4782
h is 200
math domain error
math domain error
Replanning took: 36.995148 seconds
counter is 3578
h is 200
Replanning took: 37.645832 seconds
counter is 2155
h is 200
Replanning took: 37.709666 seconds
counter is 2407
h is 200


In [373]:
newPathList[1]

[(8, 7, -1),
 (9, 7, -1),
 (10, 7, -1),
 (11, 7, -1),
 (12, 7, -1),
 (13, 7, -1),
 (14, 7, -1),
 (14, 8, -1),
 (14, 9, -1),
 (14, 10, -1),
 (14, 11, -1),
 (14, 12, -1),
 (14, 13, -1),
 (14, 14, -1),
 (14, 15, -1),
 (14, 16, -1),
 (13, 16, -1),
 (12, 16, -1),
 (11, 16, -1),
 (10, 16, -1),
 (9, 16, -1),
 (8, 16, -2),
 (8, 17, -2),
 (8, 18, -2),
 (8, 19, -2),
 (9, 19, -2),
 (10, 19, -2),
 (11, 19, -3)]

In [366]:
print(oldSafeProbList)
print(newSafeProbList)

[0.03, 0.03, 0.0, 0.0, 0.0, 0.09, 0.12, 0.12, 0.12]
[0.18, 0.08, 0.1, 0.19, 0.1, 0.09, 0.12, 0.12, 0.12]


In [370]:
print(oldSafeProbList)
print(newSafeProbList)

[0.09, 0.015, 0.0, 0.0, 0.01, 0.08, 0.055]
[0.12, 0.17, 0.155, 0.1, 0.095, 0.08, 0.055]


In [ ]:
# D* Lite result
0.06 0.06 0.01 0.04 0.06 0.05 0.11 0.05 (100)
0.045 0.04 0.035 0.025 0.02 0.06 0.05 

## Test code

In [215]:
safeProbSegmentList2 = [upe.pe.evaluate_segment_obsv(9+ind,
                               [routesFromNow[ind][0],
                                routesFromNow[ind][1]],
                               9+ind+1,
                               [routesFromNow[ind+1][0],
                                routesFromNow[ind+1][1]],
                               9,
                               upe.observedFireLocationArray) 
 for ind,(x,y,q) in enumerate(routesFromNowCopied[:-1])]
safeProbSegmentList2

[0.8473581213307241,
 0.6096579476861167,
 0.6948640483383686,
 0.6109589041095891,
 0.7465753424657534,
 0.7075471698113207,
 0.6965944272445821,
 0.7763578274760383,
 0.8532934131736527,
 0.9609756097560975,
 0.9487179487179487,
 0.8686679174484052,
 0.8486707566462167,
 0.825,
 0.9405940594059405,
 0.9501084598698482,
 0.9780439121756487,
 0.9147005444646098,
 0.9123809523809524,
 0.9278752436647173]

In [337]:
upe.dijkPathFinder.savedSafeProb[7,8,7,8,6]

0.0

In [193]:
costsFromNow, routesFromNow, controlsFromNow = aDP1.query_optimal_solution(9, [7,7,-1])
routesFromNow

[(7, 7, -1),
 (7, 8, -1),
 (6, 8, -1),
 (6, 9, -1),
 (5, 9, -1),
 (5, 10, -1),
 (5, 11, -1),
 (5, 12, -1),
 (5, 13, -1),
 (5, 14, -1),
 (5, 15, -1),
 (5, 16, -1),
 (6, 16, -1),
 (7, 16, -1),
 (8, 16, -2),
 (8, 17, -2),
 (8, 18, -2),
 (8, 19, -2),
 (9, 19, -2),
 (10, 19, -2),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3)]

In [338]:
_pathSegment, searchNodes, lastNode = MaxProbAstar(upe.M,upe.N,upe.Wall,monteCarloFireMap,startTime = upe.T, xT=xT,yT=yT,observedFireLocationArray = upe.observedFireLocationArray).astar(tuple([upe.xT,upe.yT]),tuple(loc2))

TypeError: cannot unpack non-iterable NoneType object

In [124]:
pathSegment = list(_pathSegment)

In [125]:
pathSegment

[(8, 7), (8, 8), (7, 8), (6, 8)]

In [112]:
[[pathSegment[ind+1][0],pathSegment[ind+1][1]] for ind,(x,y) in enumerate(pathSegment[:-1])]

[[8, 8], [7, 8], [6, 8]]

In [126]:
safeProbSegmentList = [upe.pe.evaluate_segment_obsv(upe.T+ind,[pathSegment[ind][0],pathSegment[ind][1]],upe.T+ind+1,[pathSegment[ind+1][0],pathSegment[ind+1][1]],upe.T,upe.observedFireLocationArray) for ind,(x,y) in enumerate(pathSegment[:-1])]
print(safeProbSegmentList)

Corresponding episode number: 302
safeProb from [8, 7] to [8, 8] is 0.946843853820598
Corresponding episode number: 302
safeProb from [8, 8] to [7, 8] is 0.856140350877193
Corresponding episode number: 302
safeProb from [7, 8] to [6, 8] is 0.6532258064516129
[0.946843853820598, 0.856140350877193, 0.6532258064516129]


In [129]:
safeProbSegment = functools.reduce(lambda a,b:a*b, safeProbSegmentList)
print(safeProbSegment)
-math.log(safeProbSegment)

0.5295252384524702


0.635774450388272